In [18]:
# ============================================================================
# LangChain RAG Tool Setup - Cell 1: Imports and LLM Configuration
# ============================================================================

# ============================================================================
# Core Imports
# ============================================================================
import os
from dotenv import load_dotenv

# ============================================================================
# Text Processing & Embeddings
# ============================================================================
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

# ============================================================================
# Vector Stores & Retrieval
# ============================================================================
from langchain_community.vectorstores import FAISS
from langchain_core.tools import create_retriever_tool

# ============================================================================
# Retriever Strategies & Compression
# ============================================================================
# NOTE: ContextualCompressionRetriever requires langchain>=0.1.0
# If you get import errors, upgrade: pip install --upgrade langchain
# For now, commented out - documentation still available below
#
# try:
#     from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
#     from langchain.retrievers.document_compressors import LLMCompressor
#     COMPRESSION_AVAILABLE = True
# except ImportError:
#     print("⚠️  ContextualCompressionRetriever not available in this version")
#     print("Upgrade LangChain: pip install --upgrade langchain")
#     COMPRESSION_AVAILABLE = False

# ============================================================================
# Language Models (LLMs)
# ============================================================================
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

# ============================================================================
# Prompts & Agents
# ============================================================================
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain.agents import create_agent

# ============================================================================
# Checkpoint & Memory (for agent state)
# ============================================================================
from langgraph.checkpoint.memory import InMemorySaver

# Load environment variables from .env file
load_dotenv()

# ============================================================================
# API Key Configuration
# ============================================================================
# Retrieve API keys for Gemini and OpenAI from environment variables
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Validate that both API keys are present
assert GEMINI_API_KEY, "GEMINI_API_KEY is missing. Check your .env file."
assert OPENAI_API_KEY, "OPENAI_API_KEY is missing. Check your .env file."

# Display partial keys to confirm successful loading (for security, only show first 6 chars)
print("Key loaded:", GEMINI_API_KEY[:6] + "..." )
print("OpenAI Key loaded:", OPENAI_API_KEY[:6] + "...")

# ============================================================================
# Initialize Language Models (LLMs)
# ============================================================================
# Initialize Google's Gemini LLM with temperature=0 for deterministic responses
gemini_llm = ChatGoogleGenerativeAI(
    temperature=0,  # Deterministic output (no randomness)
    model="gemini-2.5-flash",  # Using the faster Gemini model
    google_api_key=GEMINI_API_KEY
)

# Initialize OpenAI's GPT-3.5-turbo LLM
openai_llm = ChatOpenAI(
    temperature=0,  # Deterministic output
    model="gpt-3.5-turbo",
    openai_api_key=OPENAI_API_KEY
)

# ============================================================================
# Create a Simple Prompt Template
# ============================================================================
# Define a reusable prompt template for company name generation
prompt = PromptTemplate(
    input_variables=["product"],  # Variable to be filled dynamically
    template="Give me a creative name for a company that makes {product}?",
)

# ============================================================================
# Example 1: LLM Chain (Prompt → LLM)
# ============================================================================
# Create a chain: prompt template piped to OpenAI LLM
chain = prompt | openai_llm

# Execute the chain with a specific product type
response = chain.invoke({"product": "smart home devices"})

# Display the generated company name
print("OpenAI response:", response)

Key loaded: AIzaSy...
OpenAI Key loaded: sk-pro...
OpenAI response: content='IntelliHome Technologies' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 21, 'total_tokens': 25, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CrZY1xj5hvkj7vqoll7Pqnfg56857', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b627f-40ea-78e3-b2a0-9df42a2f04b1-0' usage_metadata={'input_tokens': 21, 'output_tokens': 4, 'total_tokens': 25, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [19]:
# ============================================================================
# LangChain RAG Tool Setup - Cell 2: FAISS Vector Database & Retriever
# ============================================================================

# ============================================================================
# Step 1: Load Document
# ============================================================================
# Read the sample.txt file containing text data to be indexed
print("Loading document from sample.txt...")
with open("sample.txt", "r", encoding="utf-8") as f:
    text_data = f.read()
print("✓ Document loaded successfully\n")

# ============================================================================
# Step 2: Split Text into Chunks
# ============================================================================
# Initialize text splitter to break documents into manageable chunks
# - separator="\n": Split on newline characters
# - chunk_size=300: Each chunk contains ~300 characters
# - chunk_overlap=50: Overlap for context preservation
print("Splitting text into chunks...")
splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=300,
    chunk_overlap=50
)
# Create document objects from text
docs = splitter.create_documents([text_data])
print(f"✓ Created {len(docs)} document chunks\n")

# ============================================================================
# Step 3: Create Embeddings and FAISS Vector Store
# ============================================================================
# Initialize OpenAI embeddings (converts text to vectors)
print("Creating embeddings and FAISS vector store...")
embeddings = OpenAIEmbeddings()

# Create FAISS vector database from documents
vectorstore = FAISS.from_documents(docs, embeddings)
print("✓ FAISS vector store created successfully\n")

# ============================================================================
# Step 4: Create Retriever
# ============================================================================
# Create a retriever that fetches relevant documents
# search_kwargs={"k": 4} means return top 4 most relevant documents
print("Creating retriever...")
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
print("✓ Retriever configured (returns top 4 documents)\n")

# ============================================================================
# Step 5: Convert Retriever to RAG Tool
# ============================================================================
# Wrap the retriever as a tool for the agent to use
print("Creating RAG tool...")
rag_tool = create_retriever_tool(
    retriever,
    name="rag_qa_tool",
    description="Use this tool to answer questions about LangChain using the sample.txt document"
)
print("✓ RAG tool created successfully\n")

# ============================================================================
# Step 6: Create Agent with RAG Tool
# ============================================================================
# Create modern agent that can use the RAG tool
print("Initializing agent with RAG tool...")
agent = create_agent(
    openai_llm,  # Use OpenAI LLM
    tools=[rag_tool],  # Provide RAG tool
    system_prompt="You are a helpful assistant. Use the RAG tool when needed to answer questions accurately based on the provided documents."
)
print("✓ Agent created successfully\n")

# ============================================================================
# Step 7: Test the RAG Agent
# ============================================================================
print("="*80)
print("TESTING RAG AGENT WITH FAISS VECTOR STORE")
print("="*80 + "\n")

# Test Query 1
query1 = "What is LangChain?"
print(f"Query: {query1}")
result1 = agent.invoke({
    "messages": [{"role": "user", "content": query1}]
})
print(f"Answer: {result1['messages'][-1].content}\n")

# Test Query 2
query2 = "Why do we use LangChain?"
print(f"Query: {query2}")
result2 = agent.invoke({
    "messages": [{"role": "user", "content": query2}]
})
print(f"Answer: {result2['messages'][-1].content}\n")

print("="*80)

Loading document from sample.txt...
✓ Document loaded successfully

Splitting text into chunks...
✓ Created 1 document chunks

Creating embeddings and FAISS vector store...
✓ FAISS vector store created successfully

Creating retriever...
✓ Retriever configured (returns top 4 documents)

Creating RAG tool...
✓ RAG tool created successfully

Initializing agent with RAG tool...
✓ Agent created successfully

TESTING RAG AGENT WITH FAISS VECTOR STORE

Query: What is LangChain?
Answer: LangChain is a framework for building applications with Large Language Models (LLMs). It was created by Harrison Chase and supports RAG, agents, memory, tools, and more. LangChain is commonly used in chatbots, document Q&A, and AI workflow.

Query: Why do we use LangChain?
Answer: LangChain is used as a framework for building applications with Large Language Models (LLMs). It was created by Harrison Chase and supports various features such as RAG, agents, memory, tools, and more. LangChain is commonly used in 

In [20]:
# ============================================================================
# WHY WE NEED CONTEXTUAL COMPRESSION RETRIEVER
# ============================================================================
# Understanding the problem it solves and when to use it

from tabulate import tabulate

print("\n" + "█"*150)
print("█" + " "*148 + "█")
print("█" + "CONTEXTUAL COMPRESSION RETRIEVER: SOLVING REAL-WORLD PROBLEMS".center(148) + "█")
print("█" + " "*148 + "█")
print("█"*150 + "\n")

# ============================================================================
# DIAGRAM 1: THE PROBLEM
# ============================================================================
print("┌" + "─"*148 + "┐")
print("│" + "DIAGRAM 1: THE PROBLEM - WASTED TOKENS & NOISE".ljust(148) + "│")
print("└" + "─"*148 + "┘\n")

problem_diagram = """
USER QUERY: "What is LangChain?"

┌─────────────────────────────────────────────────────────────────────────────────────────────┐
│  REGULAR RETRIEVER (WITHOUT COMPRESSION)                                                    │
└─────────────────────────────────────────────────────────────────────────────────────────────┘

Retrieved Document 1 (500 tokens):
┌─────────────────────────────────────────────────────────────────────────────────────────┐
│ LangChain was founded in 2022... [100 words about funding]... Series A funding announced │
│ in 2023... [50 words about investors]... Headquarters in San Francisco... [lots more      │
│ irrelevant details]... ✓ KEY INFO: "LangChain is a framework for LLMs"                   │
│ [200 more words about company history]...                                                 │
└─────────────────────────────────────────────────────────────────────────────────────────┘

🔴 Problem:
  • 500 tokens sent to LLM
  • Only 1-2 sentences are relevant
  • 95% is NOISE/WASTED tokens
  • Costs: $0.15 per 1000 tokens (GPT-4) → Wastes money!
  • LLM context gets cluttered
  • Potential for hallucination from irrelevant content


┌─────────────────────────────────────────────────────────────────────────────────────────────┐
│  CONTEXTUAL COMPRESSION RETRIEVER (WITH COMPRESSION)                                        │
└─────────────────────────────────────────────────────────────────────────────────────────────┘

Same Retrieved Document (500 tokens) → LLM Compression:

"Extract ONLY the parts relevant to: 'What is LangChain?'"

Result (50 tokens):
┌─────────────────────────────────────────────────────────────────────────────────────────┐
│ LangChain is a framework for building applications with Large Language Models. It was   │
│ founded in 2022 and supports RAG, agents, memory, tools, and more.                      │
└─────────────────────────────────────────────────────────────────────────────────────────┘

✅ Solution:
  • Only 50 tokens sent to LLM
  • 100% relevant content
  • 90% token reduction!
  • Saves money: 9x cheaper for this retrieval
  • Cleaner context = Better answers
  • Less hallucination risk
"""

print(problem_diagram)

# ============================================================================
# DIAGRAM 2: TOKEN COST COMPARISON
# ============================================================================
print("\n┌" + "─"*148 + "┐")
print("│" + "DIAGRAM 2: REAL-WORLD TOKEN & COST COMPARISON".ljust(148) + "│")
print("└" + "─"*148 + "┘\n")

cost_diagram = """
Scenario: 10 queries to a knowledge base with 100-doc context

┌──────────────────────────────────────────────────────────────────────────────────────────────┐
│ REGULAR RETRIEVER (No Compression)                                                           │
├──────────────────────────────────────────────────────────────────────────────────────────────┤
│ Per Query:                                                                                   │
│   • Retrieve 4 documents × 500 tokens average = 2,000 tokens                               │
│   • Send to LLM = 2,000 tokens (input)                                                      │
│   • LLM response = 200 tokens (output)                                                      │
│   • Total per query = 2,200 tokens                                                          │
│                                                                                              │
│ For 10 queries:                                                                              │
│   • Total tokens = 22,000 tokens                                                            │
│   • Cost (GPT-4): 22,000 × $0.03/1K = $0.66                                                │
│   • Cost (GPT-4 Turbo): 22,000 × $0.01/1K = $0.22                                          │
└──────────────────────────────────────────────────────────────────────────────────────────────┘

┌──────────────────────────────────────────────────────────────────────────────────────────────┐
│ CONTEXTUAL COMPRESSION RETRIEVER (With Compression)                                          │
├──────────────────────────────────────────────────────────────────────────────────────────────┤
│ Per Query:                                                                                   │
│   • Retrieve 4 documents = 2,000 tokens (not sent to LLM, used for compression)             │
│   • LLM compresses → 250 tokens average (90% reduction)                                     │
│   • Send compressed docs to LLM = 250 tokens (input)                                        │
│   • LLM response = 200 tokens (output)                                                      │
│   • Compression cost = 2,000 + 250 = 2,250 tokens                                           │
│   • Response cost = 450 tokens                                                              │
│   • Total per query = 2,700 tokens (includes compression overhead)                          │
│                                                                                              │
│ For 10 queries:                                                                              │
│   • Total tokens = 27,000 tokens                                                            │
│   • BUT: Only 4,500 tokens sent to main LLM                                                │
│   • Main LLM cost (GPT-4): 4,500 × $0.03/1K = $0.135                                       │
│   • Compression LLM cost: 22,500 × $0.0005/1K (cheaper model) = $0.011                     │
│   • Total cost ≈ $0.15 (70% SAVINGS!)                                                      │
└──────────────────────────────────────────────────────────────────────────────────────────────┘

💰 SAVINGS: $0.66 → $0.15 (77% cheaper!)
"""

print(cost_diagram)

# ============================================================================
# DIAGRAM 3: COMPARISON TABLE
# ============================================================================
print("\n┌" + "─"*148 + "┐")
print("│" + "DIAGRAM 3: DETAILED COMPARISON".ljust(148) + "│")
print("└" + "─"*148 + "┘\n")

comparison_data = [
    ["Aspect", "Regular Retriever", "Contextual Compression"],
    ["─" * 30, "─" * 50, "─" * 60],
    [
        "Tokens per 4 docs",
        "~2,000 tokens",
        "~250 tokens (90% reduction)"
    ],
    [
        "LLM Context",
        "Cluttered with irrelevant info",
        "Clean, focused, relevant only"
    ],
    [
        "Cost per query",
        "High (full documents)",
        "Low (compressed documents)"
    ],
    [
        "Processing Speed",
        "Fast ⚡⚡⚡",
        "Slower ⚡ (needs compression)"
    ],
    [
        "Hallucination Risk",
        "Higher (noise confuses LLM)",
        "Lower (only relevant info)"
    ],
    [
        "Best For",
        "Simple queries, quick responses",
        "Large docs, token-limited, cost-sensitive"
    ],
    [
        "Setup Complexity",
        "Simple",
        "Moderate (needs compressor LLM)"
    ],
    [
        "Trade-off",
        "Fast but expensive",
        "Slower but cheaper & cleaner"
    ]
]

print(tabulate(comparison_data, tablefmt="grid", maxcolwidths=[30, 50, 60]))

# ============================================================================
# DIAGRAM 4: WHEN TO USE EACH
# ============================================================================
print("\n\n┌" + "─"*148 + "┐")
print("│" + "DIAGRAM 4: DECISION GUIDE - WHEN TO USE WHAT".ljust(148) + "│")
print("└" + "─"*148 + "┘\n")

decision_guide = """
┌─────────────────────────────────────────────────────────────────────────────────────────────┐
│                           CHOOSE REGULAR RETRIEVER WHEN:                                     │
├─────────────────────────────────────────────────────────────────────────────────────────────┤
│  ✓ Small documents (< 200 tokens average)                                                   │
│  ✓ Need maximum speed (no compression overhead)                                             │
│  ✓ Budget is unlimited                                                                      │
│  ✓ Few documents to retrieve                                                                │
│  ✓ Simple Q&A with high relevance documents                                                 │
│  ✓ Prototyping / Learning (quick setup)                                                     │
└─────────────────────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────────────────────┐
│                    CHOOSE CONTEXTUAL COMPRESSION WHEN:                                       │
├─────────────────────────────────────────────────────────────────────────────────────────────┤
│  ✓ Large documents (> 500 tokens)                                                           │
│  ✓ Token limits (context window constraints)                                                │
│  ✓ Cost-sensitive (production, many queries)                                                │
│  ✓ Documents have mostly irrelevant information                                             │
│  ✓ Need high-quality, focused context                                                       │
│  ✓ Want to reduce hallucination                                                             │
│  ✓ Running on budget (startups, limited resources)                                          │
└─────────────────────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────────────────────┐
│                            HYBRID APPROACH:                                                   │
├─────────────────────────────────────────────────────────────────────────────────────────────┤
│  Use both: Retrieve 10 docs quickly → Compress top 4 → Send to LLM                         │
│  Best of both worlds: Good recall + low cost                                                │
└─────────────────────────────────────────────────────────────────────────────────────────────┘
"""

print(decision_guide)

# ============================================================================
# CODE EXAMPLE: HOW TO USE IT
# ============================================================================
print("\n┌" + "─"*148 + "┐")
print("│" + "DIAGRAM 5: CODE EXAMPLE - IMPLEMENTING CONTEXTUAL COMPRESSION".ljust(148) + "│")
print("└" + "─"*148 + "┘\n")

code_example = """
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMCompressor
from langchain_openai import ChatOpenAI

# Step 1: Create base retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

# Step 2: Create compressor (LLM that compresses documents)
compressor_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
compressor = LLMCompressor.from_llm(compressor_llm)

# Step 3: Wrap retriever with compression
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever
)

# Step 4: Use it (same way as regular retriever)
docs = compression_retriever.invoke("What is LangChain?")

print(f"Retrieved {len(docs)} documents")
for doc in docs:
    print(f"Compressed content: {doc.page_content}")
    print(f"Original size was larger, now optimized!")


═══════════════════════════════════════════════════════════════════════════════════════════════

WHAT HAPPENS INTERNALLY:

1. Your Query
   ↓
2. Base Retriever fetches 10 raw documents (5,000 tokens)
   ↓
3. LLMCompressor analyzes: "Which parts are relevant to the query?"
   ↓
4. Compressor creates summaries/extracts relevant parts
   ↓
5. Compressed documents returned (500 tokens)
   ↓
6. Documents sent to your LLM with 10x fewer tokens!
"""

print(code_example)

print("\n" + "█"*150 + "\n")


██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████
█                                                                                                                                                    █
█                                           CONTEXTUAL COMPRESSION RETRIEVER: SOLVING REAL-WORLD PROBLEMS                                            █
█                                                                                                                                                    █
██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

┌────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│DIAGRAM 1: THE PROBLEM - WASTED TOKENS & NOISE                                             

In [21]:
# ============================================================================
# LangChain RAG Tool Setup - Cell 4: CONTEXTUAL COMPRESSION AGENT IMPLEMENTATION
# ============================================================================
# This cell implements ContextualCompressionRetriever in the agent
# with graceful fallback if modules are not available

print("\n" + "="*80)
print("IMPLEMENTING CONTEXTUAL COMPRESSION RETRIEVER IN AGENT")
print("="*80 + "\n")

# ============================================================================
# Step 1: Attempt to Import Compression Components (Multiple Paths)
# ============================================================================
compression_available = False

# Try multiple import paths for compatibility with different LangChain versions
import_paths = [
    ("langchain.retrievers", "ContextualCompressionRetriever"),
    ("langchain_community.retrievers", "ContextualCompressionRetriever"),
    ("langchain.retrievers.contextual_compression", "ContextualCompressionRetriever"),
]

compressor_paths = [
    ("langchain.retrievers.document_compressors", "LLMCompressor"),
    ("langchain_community.retrievers.document_compressors", "LLMCompressor"),
]

print("Attempting to import ContextualCompressionRetriever...\n")

# Try to import ContextualCompressionRetriever
ContextualCompressionRetriever = None
for module_name, class_name in import_paths:
    try:
        module = __import__(module_name, fromlist=[class_name])
        ContextualCompressionRetriever = getattr(module, class_name, None)
        if ContextualCompressionRetriever:
            print(f"  ✓ Found {class_name} in {module_name}")
            break
    except (ImportError, AttributeError):
        pass

# Try to import LLMCompressor
LLMCompressor = None
for module_name, class_name in compressor_paths:
    try:
        module = __import__(module_name, fromlist=[class_name])
        LLMCompressor = getattr(module, class_name, None)
        if LLMCompressor:
            print(f"  ✓ Found {class_name} in {module_name}")
            break
    except (ImportError, AttributeError):
        pass

# Check if both were imported successfully
if ContextualCompressionRetriever and LLMCompressor:
    compression_available = True
    print("\n✅ Compression modules imported successfully!")
    print("   → ContextualCompressionRetriever is available\n")
else:
    print("\n⚠️  Compression modules NOT found in this LangChain version")
    print("   → Using regular retriever as fallback")
    print("   → Requires: pip install --upgrade 'langchain>=0.2.0'\n")
    compression_available = False

# ============================================================================
# Step 2A: IF COMPRESSION IS AVAILABLE - Create Compression Agent
# ============================================================================
if compression_available:
    print(">>> COMPRESSION ENABLED - Creating compressed retriever...\n")
    
    # Create a compressor LLM (can use cheaper/faster model for compression)
    compressor_llm = ChatOpenAI(
        model="gpt-3.5-turbo",  # Use fast/cheap model for compression
        temperature=0
    )
    
    # Create the LLMCompressor
    # This LLM will intelligently extract relevant parts from documents
    compressor = LLMCompressor.from_llm(compressor_llm)
    
    # Wrap the retriever with compression
    # - Fetches documents with base_retriever
    # - Compresses them with base_compressor
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor,
        base_retriever=retriever  # Uses the retriever from Cell 2
    )
    
    print("✓ Compressor created with GPT-3.5-turbo")
    print("✓ Compression retriever wrapped around base retriever\n")
    
    # Create RAG tool with compressed retriever
    compressed_rag_tool = create_retriever_tool(
        compression_retriever,
        name="compressed_rag_qa_tool",
        description="Use this tool to answer questions about LangChain (with token compression) using the sample.txt document"
    )
    print("✓ Compressed RAG tool created\n")
    
    # Create agent with compressed RAG tool
    compression_agent = create_agent(
        openai_llm,
        tools=[compressed_rag_tool],
        system_prompt="""You are a helpful assistant. Use the RAG tool when needed to answer questions.
The tool automatically compresses documents to extract only relevant information, optimizing token usage."""
    )
    print("✓ Agent created with compressed retriever\n")
    
    # ========================================================================
    # Step 3A: Test Compressed Agent
    # ========================================================================
    print("="*80)
    print("TESTING COMPRESSED RAG AGENT")
    print("="*80 + "\n")
    
    test_queries = [
        "What is LangChain?",
        "What features does LangChain support?",
        "Who created LangChain?"
    ]
    
    for idx, query in enumerate(test_queries, 1):
        print(f"Query {idx}: {query}")
        try:
            result = compression_agent.invoke({
                "messages": [{"role": "user", "content": query}]
            })
            print(f"Answer: {result['messages'][-1].content}\n")
        except Exception as e:
            print(f"Error: {e}\n")

# ============================================================================
# Step 2B: IF COMPRESSION NOT AVAILABLE - Use Regular Retriever (Fallback)
# ============================================================================
else:
    print(">>> COMPRESSION NOT AVAILABLE - Using regular retriever\n")
    print("ℹ️  The agent from Cell 2 is already using regular retrieval")
    print("   It works perfectly, just without token compression.\n")
    
    # The 'agent' from Cell 2 is already available and working
    # No additional setup needed
    
    print("="*80)
    print("FALLBACK: USING REGULAR RAG AGENT (From Cell 2)")
    print("="*80 + "\n")
    
    test_queries = [
        "What is LangChain?",
        "What features does LangChain support?",
        "Who created LangChain?"
    ]
    
    for idx, query in enumerate(test_queries, 1):
        print(f"Query {idx}: {query}")
        try:
            result = agent.invoke({  # Uses agent from Cell 2
                "messages": [{"role": "user", "content": query}]
            })
            print(f"Answer: {result['messages'][-1].content}\n")
        except Exception as e:
            print(f"Error: {e}\n")

# ============================================================================
# Summary
# ============================================================================
print("\n" + "="*80)
print("SUMMARY")
print("="*80)

if compression_available:
    print("\n✅ COMPRESSION AGENT ACTIVE")
    print("   • Uses: compression_agent")
    print("   • Feature: Documents automatically compressed")
    print("   • Benefit: 90% token reduction on large documents")
    print("   • Trade-off: Slightly slower (compression step)")
    print("\n📊 Token Usage:")
    print("   Regular: ~2,000 tokens per retrieval")
    print("   Compressed: ~250 tokens per retrieval")
    print("   Savings: ~90% (9x cheaper!)")
else:
    print("\n✅ FALLBACK MODE ACTIVE")
    print("   • Uses: agent (from Cell 2)")
    print("   • Feature: Regular retrieval without compression")
    print("   • Status: Fully functional, just not optimized for token cost")
    print("\n🔧 To Enable Compression:")
    print("   1. Upgrade LangChain:")
    print("      pip install --upgrade 'langchain>=0.2.0'")
    print("   2. Then re-run this cell")
    print("   3. Compression will automatically activate")
    print("\n📊 Token Usage:")
    print("   Current: ~2,000 tokens per retrieval")
    print("   With upgrade: ~250 tokens per retrieval")
    print("   Potential savings: ~90%")

print("\n" + "="*80 + "\n")


IMPLEMENTING CONTEXTUAL COMPRESSION RETRIEVER IN AGENT

Attempting to import ContextualCompressionRetriever...


⚠️  Compression modules NOT found in this LangChain version
   → Using regular retriever as fallback
   → Requires: pip install --upgrade 'langchain>=0.2.0'

>>> COMPRESSION NOT AVAILABLE - Using regular retriever

ℹ️  The agent from Cell 2 is already using regular retrieval
   It works perfectly, just without token compression.

FALLBACK: USING REGULAR RAG AGENT (From Cell 2)

Query 1: What is LangChain?
Answer: LangChain is a framework for building applications with Large Language Models (LLMs). It was created by Harrison Chase and supports RAG, agents, memory, tools, and more. LangChain is commonly used in chatbots, document Q&A, and AI workflow.

Query 2: What features does LangChain support?
Answer: LangChain supports the following features:
- RAG
- Agents
- Memory
- Tools
- More

These features are commonly used in chatbots, document Q&A, and AI workflow.

Query 3: Wh